# Introduction to Portfolio Theory: Risk Measures #

After studying benchmarks, Nancy concluded that she was somewhat fortunate to have chosen Nvidia stock as her first investment. Now, she wants to invest in an ETF because she believes it can help her maintain a medium-to-long-term investment strategy, as she is not ready for daily trading. Nancy wants to be more careful when choosing an ETF, so she compares three ETFs (SPY, IWY, and SPXL) and their risks.

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Statistics
from scipy.stats import norm

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from data_downloader import get_market_data

# Visualization
import matplotlib.pyplot as plt

In [2]:
# Import Data
df_1 = get_market_data(
    ticker='SPY', 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=True
)

df_1

In [3]:
# Get more data

# Tech ETF
df_2 = get_market_data(
    ticker='IWY', 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=True
)

# SP500 x3
df_3 = get_market_data(
    ticker='SPXL', 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=True
)

In [4]:
# Create the data we are going to use
data = pd.DataFrame()
data['SPY'] = df_1['returns']
data['IWY'] = df_2['returns']
data['SPXL'] = df_3['returns']

data

In [5]:
# 1. Annualized Volatility (Standard Deviation)
volatility = data.std() * np.sqrt(252)

print("Annualized Volatility:\n")
print(volatility)

In [38]:
# 1.5 Annualized Variance (Standard Deviation)

variance = data.var() * 252

print("Annualized Variance:\n")
print(variance)

In [6]:
# 2. Covariance Matrix
covariance = data.cov() * 252  # Annualized

print("Covariance Matrix:\n")
print(covariance)

In [7]:
# 3. Correlations
correlations = data.corr()

print("Correlation Matrix:\n")
print(correlations)

In [8]:
# 4. Value at Risk
confidence_level = 0.95

z_score = norm.ppf(1 - confidence_level)

VaR_95 = data.mean() + z_score * data.std()
VaR_95 = VaR_95 * np.sqrt(252)  # Annualized

print("Annualized Value at Risk (VaR) at 95% confidence level:\n")
print(VaR_95)

In [12]:
# Download the Benchmark
benchmark = pd.read_csv(r'..\additional_data\sp500.csv')
benchmark.set_index('Date', inplace=True)
benchmark.index = pd.to_datetime(benchmark.index)

benchmark

In [13]:
# Calculate the benchmark returns
benchmark_rets = benchmark['sp_500']

In [14]:
# Calculate the Excess Returns
excess_return = data.subtract(benchmark_rets.squeeze(), axis=0)

excess_return

In [15]:
# Now calculate the Tracking Error (which is the Std of the Excess Returns)
tracking_error = excess_return.std() * np.sqrt(252)

print("\nTracking Error:")
print(tracking_error)

In [16]:
#Let us visualize the Risk Measures
fig, ax = plt.subplots(figsize=(10, 6))
volatility.plot(kind='bar', color='skyblue', label='Volatility (Risk)', alpha=0.5)
tracking_error.plot(kind='bar', color='lightgreen', label='Tracking Error', alpha=0.5)
VaR_95.plot(kind='bar', color='salmon', label='VaR 95%', alpha=0.7)
plt.title('Risk Measures')
plt.ylabel('Value')
plt.legend()
plt.show()

In [17]:
# Naturally, the SPXL is more risky that the other ETFs
mean = data.mean() * 252

print("Annualized Mean Returns:")
print(mean)

In [18]:
# Risk Adjusted Returns
eff_ratio = mean/volatility

print("Risk Adjusted Returns:")
print(eff_ratio)

In [19]:
#Let us visualize
fig, ax = plt.subplots(figsize=(10, 6))
eff_ratio.plot(kind='bar', color='lightpink', label='Risk Adjusted Returns')
ax.axhline(y=1, color='black', linestyle='--')
plt.title('Risk Adjusted Returns')
plt.ylabel('Value')
plt.legend()
plt.show()

In [20]:
# Calculate the Risk Adjusted Returns of the Benchmark (Risk Aversion Coefficient)

mean_bench = benchmark_rets.mean() * 252

var_bench = benchmark_rets.var() * 252

eff_rate_bench = mean_bench/var_bench

eff_rate_bench